# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP. 

First we import the necessary packages and create the global variables.

In [1]:
import os, math, random, pickle, time
import numpy as np
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler,EarlyStopping
from keras.models import Sequential, Model, load_model
from keras.layers.pooling import AveragePooling1D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation, Input, merge, Conv2D, Reshape, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
import keras
from sklearn import preprocessing
from keras import backend as K
from keras import metrics
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import CMAPSAuxFunctions

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Constants

In [2]:
FeatureN = 14
nb_epoch = 250
batch_size = 512
FilterN = 10
FilterL = 10
rmse,sco,tm = [], [], []

ConstRUL = 125
TW = 30
Dataset = '1'

# Retrieve the train and test data from the files

In [3]:
############ training samples ##################################

setTrain = {'1':100, '2':260, '3':100, '4':248}
setTest = {'1':100, '2':259, '3':100, '4':248}
nTrain = setTrain[Dataset]
nTest = setTest[Dataset]

data = [] 
for line in open("../CMAPSSData/train_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
data = min_max_scaler.fit_transform(data)   #Why scale the data with all of the "missing" rows.
num=[]
for i in range(nTrain):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    num.append(tmp)
num=np.array(num)

label=[]
for i in range(nTrain):
    label.append([])
    length = len(num[i])
    for j in range(length):
        label[i].append(ConstRUL if length-j-1>=ConstRUL else length-j-1)
label = np.array(label)

samples,targets,noofsample = [],[],[]
for i in range(nTrain):
    noofsample.append(len(num[i])-TW+1)
    for j in range(noofsample[-1]):
        samples.append(num[i][j:j+TW,:])
        targets.append(label[i][j+TW-1])
samples = np.array(samples)
targets = np.array(targets)

################## testing data ###########################
data = [] 
for line in open("../CMAPSSData/test_FD00"+Dataset+".txt"):
    data.append(line.split())
data=np.array(data)
data = np.cast['float64'](data)
data_copy = data
data = min_max_scaler.transform(data)  #Why scale the data with all of the "missing" rows.
numt=[]
for i in range(nTest):
    tmp = data[np.where(data_copy[:,0]==i+1),:][0][:, np.array([6,7,8,11,12,13,15,16,17,18,19,21,24,25])]
    numt.append(tmp)
numt=np.array(numt)

samplet, count_miss = [],[]
for i in range(nTest):
    if len(numt[i])>=TW:
        samplet.append(numt[i][-TW:,:])
    else:
        count_miss.append(i)
samplet = np.array(samplet)

labelt = [] 
for line in open("../CMAPSSData/RUL_FD00"+Dataset+".txt"):
    labelt.append(line.split())
labelt = np.cast['int32'](labelt)
labelnew = []
for i in range(nTest):
    if i not in count_miss:
        #labelnew.append(labelt[i][0])
        labelnew.append(labelt[i][0] if labelt[i][0]<=ConstRUL else ConstRUL)
labelt = labelnew
labelt=np.array(labelt)

'''seed = 2222
np.random.seed(seed)
np.random.shuffle(samples)
np.random.seed(seed)
np.random.shuffle(targets)
samplet = samplet[np.argsort(labelt)]
labelt = labelt[np.argsort(labelt)]'''

'seed = 2222\nnp.random.seed(seed)\nnp.random.shuffle(samples)\nnp.random.seed(seed)\nnp.random.shuffle(targets)\nsamplet = samplet[np.argsort(labelt)]\nlabelt = labelt[np.argsort(labelt)]'

In [4]:
print("Training data")
print(samples.shape)
print(targets.shape)
print("Testing data")
print(samplet.shape)
print(labelt.shape)
print("Training data")
print(samples[-5:,:])
print(targets[-5:])
print("Testing data")
print(samplet[-5:,:])
print(labelt[-5:])

Training data
(17731, 30, 14)
(17731,)
Testing data
(100, 30, 14)
(100,)
Training data
[[[ 0.42168675  0.12579028  0.17049291 ...  0.33333333 -0.27131783
   -0.07953604]
  [ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  ...
  [ 0.51204819  0.14453891  0.52464551 ...  0.         -0.62790698
   -0.34217067]
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]]

 [[ 0.06024096 -0.02419882  0.37812289 ...  0.16666667 -0.27131783
    0.01712234]
  [-0.01204819 -0.02419882  0.47467927 ...  0.16666667 -0.11627907
   -0.11101906]
  [-0.09638554 -0.02419882  0.53376097 ...  0.         -0.24031008
   -0.2761668 ]
  ...
  [ 0.3253012   0.26444299  0.67623228 ...  0.         -1.
   -0.17674676]
  [ 0.37349398  0.17462394  0.5658339  ...  0.5        -0.45736434
   -0.78099972]
  

# Keras model

CNN model. The model is Dense(ReLU, 100)->Dense(ReLu, 100)->Dense(Linear, 1)

In [5]:
def RULCNNModel(TW, FeatureN):
    
    input_layer = Input(shape=(TW, FeatureN))
    y = Reshape((TW, FeatureN, 1), input_shape=(TW, FeatureN, ),name = 'Reshape')(input_layer)

    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C1')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C2')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C3')(y)
    y = Conv2D(FilterN, FilterL, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='C4')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C5')(y)
    #y = Convolution2D(FilterN, FilterL, 1, border_mode='same', init='glorot_normal', activation='tanh', name='C6')(y)
    
    y = Conv2D(1, 3, 1, border_mode='same', kernel_initializer='glorot_normal', activation='tanh', name='Clast')(y)  
    
    y = Reshape((TW,14))(y)
    y = Flatten()(y)
    y = Dropout(0.5)(y)
    
    #y = Dense(100, activation='tanh', init='glorot_normal', activity_regularizer=keras.regularizers.l2(0.01),)(y)
    y = Dense(100,activation='tanh', kernel_initializer='glorot_normal', name='fc')(y)
    y = Dense(1)(y)
    
    model = Model(inputs = input_layer, outputs = y, name='RUL_CNN_Model')
    
    return model

# Fit the keras model

Fit the Keras model to the data and determine its performance.

In [7]:
opt = Adam(lr=0, beta_1=0.5)
#DCNN = Model([input_layer], [y])
DCNN = RULCNNModel(TW, FeatureN)
#DCNN.compile(loss=get_score,optimizer=opt)
DCNN.compile(loss='mean_squared_error',optimizer=opt)
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)


startTime = time.clock()
history = DCNN.fit(samples, targets,nb_epoch=nb_epoch, batch_size=batch_size,verbose=1, 
                   validation_data=(samplet, labelt), callbacks=[lrate])
endTime = time.clock()

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C1", activation="tanh", kernel_initializer="glorot_normal", padding="same")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C2", activation="tanh", kernel_initializer="glorot_normal", padding="same")`
  import sys
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C3", activation="tanh", kernel_initializer="glorot_normal", padding="same")`
  
C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (10, 1), name="C4", activation="tanh", kernel_initializer="gl

Train on 17731 samples, validate on 100 samples
Epoch 1/250
17731/17731 [==============================] - 1s 80us/step - loss: 6477.6097 - val_loss: 4985.7393
Epoch 2/250
17731/17731 [==============================] - 0s 22us/step - loss: 5653.3891 - val_loss: 4567.2373
Epoch 3/250
17731/17731 [==============================] - 0s 22us/step - loss: 5211.3580 - val_loss: 4197.2041
Epoch 4/250
17731/17731 [==============================] - 0s 22us/step - loss: 4812.7686 - val_loss: 3858.3606
Epoch 5/250
17731/17731 [==============================] - 0s 22us/step - loss: 4446.7817 - val_loss: 3547.8027
Epoch 6/250
17731/17731 [==============================] - 0s 22us/step - loss: 4109.5075 - val_loss: 3260.2002
Epoch 7/250
17731/17731 [==============================] - 0s 22us/step - loss: 3797.7512 - val_loss: 2996.3311
Epoch 8/250
17731/17731 [==============================] - 0s 22us/step - loss: 3509.0816 - val_loss: 2751.6165
Epoch 9/250
17731/17731 [==============================]

17731/17731 [==============================] - 0s 23us/step - loss: 188.0168 - val_loss: 179.9889
Epoch 75/250
17731/17731 [==============================] - 0s 22us/step - loss: 209.8040 - val_loss: 170.7365
Epoch 76/250
17731/17731 [==============================] - 0s 22us/step - loss: 181.0095 - val_loss: 160.1453
Epoch 77/250
17731/17731 [==============================] - 0s 22us/step - loss: 187.8749 - val_loss: 151.4225
Epoch 78/250
17731/17731 [==============================] - 0s 22us/step - loss: 177.4997 - val_loss: 215.1797
Epoch 79/250
17731/17731 [==============================] - 0s 22us/step - loss: 187.7312 - val_loss: 154.0558
Epoch 80/250
17731/17731 [==============================] - 0s 22us/step - loss: 176.9077 - val_loss: 171.2271
Epoch 81/250
17731/17731 [==============================] - 0s 23us/step - loss: 191.0667 - val_loss: 376.8734
Epoch 82/250
17731/17731 [==============================] - 0s 22us/step - loss: 202.7675 - val_loss: 169.1406
Epoch 83/250
1

17731/17731 [==============================] - ETA: 0s - loss: 120.061 - 0s 22us/step - loss: 119.7644 - val_loss: 154.6385
Epoch 221/250
17731/17731 [==============================] - 0s 22us/step - loss: 118.9979 - val_loss: 156.2561
Epoch 222/250
17731/17731 [==============================] - 0s 23us/step - loss: 120.9058 - val_loss: 154.4792
Epoch 223/250
17731/17731 [==============================] - 0s 22us/step - loss: 118.7322 - val_loss: 154.4244
Epoch 224/250
17731/17731 [==============================] - 0s 22us/step - loss: 118.9382 - val_loss: 160.5224
Epoch 225/250
17731/17731 [==============================] - 0s 22us/step - loss: 120.2956 - val_loss: 154.7111
Epoch 226/250
17731/17731 [==============================] - 0s 22us/step - loss: 120.4925 - val_loss: 158.9643
Epoch 227/250
17731/17731 [==============================] - 0s 22us/step - loss: 120.3448 - val_loss: 159.0265
Epoch 228/250
17731/17731 [==============================] - 0s 22us/step - loss: 118.7513 -

In [11]:
#Evaluate the model
score = DCNN.evaluate(samplet, labelt)
y_pred = DCNN.predict(samplet)
healtScore = CMAPSAuxFunctions.compute_health_score(labelt, y_pred)

print("Root Square Mean Error score: {}".format(np.sqrt(score)))
print("Health score: {}".format(healtScore))
print("Elapsed time: {}".format(endTime - startTime))

100/100 [==============================] - 0s 80us/step
Root Square Mean Error score: 12.56286816109703
Health score: [270.5371]
Elapsed time: 99.38800193682309
